In [1]:
from torch.utils.data import Dataset, DataLoader
from matplotlib import pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np
import torch
import os

In [2]:
class wordpicDataset(Dataset):
    
    def __init__(self, root_dir, train=True, transform=None):
        """
        read csv's and descriptions directory to extract
        raw data in (description, resnet_embedding, image_label) format
        """
        
        self.transform = transform

        if train:
            resnet = pd.read_csv(os.path.join(root_dir, \
                                'features_train/features_resnet1000_train.csv'),header=None)
            
            # strip file directory and .jpg extension, extract only image number for indexing
            idx = resnet.iloc[:,0].str.split('/')[:].apply(pd.Series).iloc[:,1] \
                        .str.split('.').apply(pd.Series).iloc[:,0].astype('int32')
            resnet.iloc[:,0] = idx
            resnet = resnet.sort_values(by=0)
            self.resnet = resnet
            
            # TODO: explore intermediate resnet feature map
            self.resnet_inter = pd.read_csv(os.path.join(root_dir, \
                                'features_train/features_resnet1000intermediate_train.csv'),header=None)
            
            # os.walk doesn't supply texts in numeric order
            texts = ['None']*10000
            for root, directory, files in os.walk(os.path.join(root_dir,'descriptions_train')):
                for file in files:
                    d = os.path.join(root_dir,'descriptions_train')
                    with open(os.path.join(d,file),'r') as f:
                        text = f.read().replace('\n','')
                        txt_loc = int(file.split('.')[0])
                        texts[txt_loc] = text
                        
            # return (description, resnet_embedding, image_label) 
            self.raw_data = []
                    
            for i in range(10000):
                # store image label as int 
                # TODO: is image file path more convenient?
                self.raw_data.append((texts[i],self.resnet.iloc[i,1:],i))

        else:
            
            resnet = pd.read_csv(os.path.join(root_dir, \
                                'features_test/features_resnet1000_test.csv'),header=None)
            
            idx = resnet.iloc[:,0].str.split('/')[:].apply(pd.Series).iloc[:,1] \
                        .str.split('.').apply(pd.Series).iloc[:,0].astype('int32')
            resnet.iloc[:,0] = idx
            resnet = resnet.sort_values(by=0)
            self.resnet = resnet
            
            self.resnet_inter = pd.read_csv(os.path.join(root_dir, \
                                'features_test/features_resnet1000intermediate_test.csv'),header=None)
            self.transform = transform
            
            texts = ['None']*2000
            for root, directory, files in os.walk(os.path.join(root_dir,'descriptions_test')):
                for file in files:
                    d = os.path.join(root_dir,'descriptions_test')
                    with open(os.path.join(d,file),'r') as f:
                        text = f.read().replace('\n','')
                        txt_loc = int(file.split('.')[0])
                        texts[txt_loc] = text
            
            # return (description, resnet_embedding) 
            self.raw_data = []

            for i in range(2000):
                self.raw_data.append((texts[i],self.resnet.iloc[i,1:]))
        
        
    def __len__(self):
        return len(self.raw_data)
    
    
    def __getitem__(self,idx):
        
        if type(idx) == torch.tensor:
            idx = idx.item()
            
        # TODO: embedding transforms
        if self.transform:
            pass
        
        # check resnet dim
        assert self.raw_data[idx][1].shape == (1000,), self.raw_data[idx][1].shape
        return self.raw_data[idx]

In [ ]:
train = wordpicDataset('data/')
test = wordpicDataset('data/',train=False)

In [ ]:
# see if the data is formatted correctly
print(train[0][0])
display(Image.open('data/images_train/0.jpg'))

# find first resnet feature vector to compare
resnet = pd.read_csv(os.path.join('data/', \
                                'features_train/features_resnet1000_train.csv'),header=None)
idx = resnet.iloc[:,0].str.split('/')[:].apply(pd.Series).iloc[:,1] \
            .str.split('.').apply(pd.Series).iloc[:,0].astype('int32')
resnet.iloc[:,0] = idx
resnet = resnet.sort_values(by=0)
print("\ncorrect resnet feature: " + str(np.array_equal(resnet.iloc[0,1:],train[0][1])))

In [ ]:
# check test data
print(test[0][0])
print(test[0][1].shape)

In [ ]:
torch.manual_seed(0)

BATCH_SIZE = 128